In [1]:
!pip install web3

In [2]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [3]:
BSC_API_KEY = "V1A3UZAFFKSNQUHCWA6MYDZ31364JDBGSJ"
BSC_URL = 'https://api.bscscan.com/api'

web3ProviderURL = "https://rpc.ankr.com/bsc/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(web3ProviderURL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [4]:
factories = {
    "biswap": {
        "address" : ["0x858e3312ed3a876947ea49d572a7c42de08af7ee"],
        "pool" : []
        }
}

In [5]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": BSC_API_KEY
    }
    response = requests.post(BSC_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [6]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [7]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "pancakeswap_v3": 10000*100,
        "uniswap_v3": 10000*100,
        "vyper_contract": 10**10,
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    elif name == "fraxswap":
        fee = 10000 - float(contract.functions.fee().call())
        fee = fee / 10000
        return fee
    return None


# Try get fee from swapFee function in abi (Biswap, Nomiswap, Global LPs, Fins)
def get_fee_from_swapFee(contract, name):
    deno_dict = {
        "biswap": 1000,
        "nomiswap": 1000,
        "nomiswapStable": 100000,
        "global": 100000,
        "fins": 10000,
        "plearn": 1000
    }

    if name in deno_dict:
        swap_fee = float(contract.functions.swapFee().call())
        fee = swap_fee / deno_dict[name]
        return fee
    return None


def get_fee_for_thena(contract, name):
    try:
        isStable = bool(contract.functions.stable().call())
        if isStable:
            fee = 4 / (100*100)
        else:
            fee = 18 / (100*100)
        return fee
    except Exception as e:
        return None


def get_fee(contract, name):
    fix_fee_dict = {
        "apeswap" : 2/1000,
        "pancake": 25/10000,
        "uniswap_v2": 3/1000,
        "sushiswap" : 3/1000,
        "babydoge": 100/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFee,
        get_fee_for_thena
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            return fee
    return fee

In [8]:
def get_tokens(contract):
    try:
        token0 = contract.functions.token0().call().lower()
        token1 = contract.functions.token1().call().lower()
        return token0, token1
    except Exception:
        return None, None

In [9]:
def get_pool_from_allPairs(factory, start, end):
    rows = []
    name = factory
    for factory_address in factories[factory]["address"]:
        factory_contract = get_contract(factory_address)
        try:
            allPairsLength = factory_contract.functions.allPairsLength().call()
            if end > allPairsLength:
                end = allPairsLength
            for i in tqdm(range(start, end)):
                try:
                    pool_address = factory_contract.functions.allPairs(i).call().lower()
                    contract = get_contract(pool_address)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "fee" : fee,
                        "contract" : pool_address,
                        "token0" : token0,
                        "token1" : token1,
                    }
                    rows.append(row)
                    time.sleep(0.2)
                except ValueError:
                    continue
        except ABIFunctionNotFound:
            continue
    pool = pd.DataFrame(rows)
    pool = pool.drop_duplicates(ignore_index = True)
    return pool

In [10]:
def get_tvl(pool):
    token_type = ['NATIVE']
    rows = []

    for i in tqdm(range(0, len(pool))):
        payload = {
            "jsonrpc": "2.0",
            "method": "ankr_getAccountBalance",
            "params": {
                "blockchain": ["bsc"],
                "walletAddress": pool.contract[i],
                "nativeFirst": True,
                "onlyWhitelisted": True,
                "pageSize": 2
            },
            "id": 1
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        response = requests.post(getAccountBalance_url, json=payload, headers=headers)
        time.sleep(1)
        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                tvl = data['result']['totalBalanceUsd']
            except KeyError:
                tvl = None
            row = {
                "name" : pool.name[i],
                "factory" : pool.factory[i],
                "tvl" : tvl,
                "fee" : pool.fee[i],
                "contract" : pool.contract[i],
                "token0" : pool.token0[i],
                "token1" : pool.token1[i],
            }

            rows.append(row)
    pool = pd.DataFrame(rows)
    pool = pool.dropna(axis=0).reset_index(drop=True)
    return pool

In [ ]:
for count in range(0, 10000):
    start = count * 1000
    end = start + 1000
    pool = get_pool_from_allPairs("biswap", start, end)
    if len(pool) <= 0:
        break
    pool = get_tvl(pool)
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

 97%|██████████████████████████████████████▋ | 968/1000 [19:48<00:39,  1.23s/it]

In [ ]:
files = [f"part{i}.csv" for i in range(count)]

In [ ]:
combined_df = pd.DataFrame()

In [ ]:
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)
    
combined_df = combined_df.drop_duplicates()
combined_df

In [ ]:
combined_df.to_csv('bsc_biswap.csv', index=False)